In [1]:
# URL Used: https://covidtracking.com/data

# Import Libraries
import pandas as pd


In [2]:
# # First dataframe checked
# csv_file = "OG_data_files/countrywide_daily_testing.csv"
# testing_df = pd.read_csv(csv_file)
# testing_df.head(50)
# sorted_df = testing_df.sort_values("Date")
# sorted_df.tail(10)

In [3]:
# Pull in CSV file using Pandas DF
testing_csv = "Resources/all_states_testing.csv"
by_state_df = pd.read_csv(testing_csv)

# Check Column header names to craft new DF
by_state_df.columns

FileNotFoundError: [Errno 2] File /Resources/all_states_testing.csv does not exist: '/Resources/all_states_testing.csv'

In [ ]:
# Craft and sort new DF to use for database
new_state_df = pd.DataFrame(by_state_df[['date', 'state','negative','negativeIncrease', 'positive','positiveIncrease', 'death','deathProbable','pending','totalTestResults']])

# Remove irrelevant states (districts)
new_state_df.drop(new_state_df.loc[new_state_df['state']=='AS'].index, inplace=True)
# new_state_df.drop(new_state_df.loc[new_state_df['state']=='DC'].index, inplace=True)
new_state_df.drop(new_state_df.loc[new_state_df['state']=='GU'].index, inplace=True)
new_state_df.drop(new_state_df.loc[new_state_df['state']=='MP'].index, inplace=True)
new_state_df.drop(new_state_df.loc[new_state_df['state']=='PR'].index, inplace=True)
new_state_df.drop(new_state_df.loc[new_state_df['state']=='VI'].index, inplace=True)



# Sort the DF by 'state' then 'date'
sorted_state_df = new_state_df.sort_values(['state', 'date'])
sorted_state_df.head()
sorted_state_df


In [ ]:
# Check if we deleted the correct number and state names
states = sorted_state_df['state']
for state in states:
#     print(state)
    if sorted_state_df['state'].nunique() == 51:
        print('Yes! We have 50 states in the df')
        break

In [ ]:
# Import CSV with states Name and Code to match up with other DF
states_csv = "Resources/states_abbrev.csv"
states_df = pd.read_csv(states_csv)
states_code_df = pd.DataFrame(states_df[['State','Code']])

# Rename column headers to merge DF's
states_code_df = states_code_df.rename(columns={'State':'state_name', 'Code':'state'})

# # Drop District of Columbia code
# # df[df.name != 'Tina']
# states_code_df = states_code_df.drop(states_code_df.index[8])

# # sort the df
# states_code_df = states_code_df.sort_values(by='state_name').reset_index(drop=True)

# # Turn back into a DF
# states_code_df = pd.DataFrame(states_code_df)

states_code_df.head()

In [ ]:
# Check number of states is correct
states = states_code_df['state']
for state in states:
#     print(state)
    if states_code_df['state'].nunique() == 51:
        print('Yes! We have 50 states in the df')
        break

In [ ]:
# Merge sorted_state_df and states_code_df in order to match sorted_state_df and @Authum's DF created
states_testing_df = pd.merge(sorted_state_df, states_code_df, on='state', how='outer')
states_testing_df = states_testing_df[['date', 'state','state_name','negative','negativeIncrease', 'positive','positiveIncrease', 'death','deathProbable','pending','totalTestResults']]

# Double check correct number of states and state name
print(states_testing_df['state_name'].nunique())
print(states_testing_df['state'].nunique())

# Verify DC is still in dataset
states_testing_df.loc[states_testing_df['state']=='DC']

#Show df one last time...
states_testing_df = states_testing_df.astype({'date': 'str'})
states_testing_df.dtypes

In [ ]:
# Date to datetime
states_testing_df['date'] = pd.to_datetime(states_testing_df['date'], format = "%Y%m%d").dt.strftime('%Y-%m-%d') 
states_testing_df

In [ ]:
# Export crafted df to new CSV file
states_testing_df.to_csv("CSV_files_created/states_testing.csv", index=False)
